In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers.trainer_utils import EvaluationStrategy
import os
import torch

In [ ]:
df = pd.read_csv("/content/prufrock_metaphor_conversation_final.csv")

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision_literal": precision[0],
        "recall_literal": recall[0],
        "f1_literal": f1[0],
        "precision_metaphor": precision[1],
        "recall_metaphor": recall[1],
        "f1_metaphor": f1[1],
    }


In [ ]:
os.environ["WANDB_DISABLED"] = "true"  # disable W&B logging

training_args = TrainingArguments(
    output_dir="./models/prufrock_roberta",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1_metaphor"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


In [ ]:
trainer.save_model("./models/prufrock_roberta")


In [ ]:
model.eval()

def predict_sentences(sentences):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
    for sent, pred in zip(sentences, predictions.cpu().numpy()):
        print(f"Sentence: {sent}\n→ Prediction: {'Metaphor' if pred == 1 else 'Literal'}\n")

In [ ]:
test_sentences = [
    "I have measured out my life with coffee spoons.",
    "He submitted the assignment on time.",
    "She's carrying emotional baggage from college.",
    "life is a river."
]
predict_sentences(test_sentences)